<a href="https://colab.research.google.com/github/CodingTomo/PyTorch-Tutorials/blob/master/PyTorch_Calculus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Calculus

Pytorch implementa alcune funzioni per il calcolo differenziale.

**Automatic differentation** (o *autodiff*) è la caratteristica principale del framework e permette di differenziare qualsiasi funzione rispetto ai suoi input in modo automatico. In pratica consente al programmatore di non preoccuparsi del calcolo del gradienti che in molti algoritmi di ottimizzazione si rendono necessari. 

In [0]:
import os

if not os.path.exists('/content/PyTorch-Tutorials'):
  !git clone https://github.com/CodingTomo/PyTorch-Tutorials #clono la repository

In [0]:
import torch

Consideriamo la funzione: $f(x) = 3 x^2 + 4$. La sua derivata vale $f^\prime(x)=6 x$. Calcoliamola struttando PyTorch.

**Osservazione**: ogni tensore ha un flag *requires_grad*. Questo flag permette di anticipare se il tensore sarà parte o meno del calcolo di un qualche gradiente.




In [0]:
# il tensore x gioca il ruolo dell'indeterminata. Non potendo gestire il calcolo simbolico, inizializiamo x a 2.0
x = torch.tensor(2.0, requires_grad=True) 
print("x = {}".format(x))

# definiamo f(x)
y = 3 * x**2  + 4
print("y = {}".format(y))

# backward() calcola la derivata di una funzione rispetto a tutte le variabili e che hanno il flag 'requires_grad'=True. 
y.backward()

# In questo caso y.backward() va a scrivere nell'attributo x.grad il valore della derivata di y in x=2
computed_gradient = x.grad
print("Il valore calcolato per la derivata è {}".format(computed_gradient))
print("Il valore atteso per la derivata è {}".format(6 * x))


Tutta la magia e la potenza di *autodiff* sta nel implemetare la **regola della catena** usata in matematica per calcolare le derivate di funzioni composte.

[Ulteriori infomazioni sulla regola della catena](https://en.wikipedia.org/wiki/Chain_rule)



---



Per derivare semplici funzioni, come $3x^2 + 4$, esistono delle regole base da seguire che sono conseguenza dello svolgimento di alcuni particolari limiti di rapporti incrementali. Queste regole base vengono concatenate e usate da PyTorch per ottenere derivate di funzioni più elaborate.

**Esempio**:

$$y = f(x) = \text{plus4}(\text{times3}(\text{square}(x))),$$

la derivata di $y$ rispetto a $x$ può essere calcolata usando la regola della catena:

$$f'(x) = \text{plus4}'(\cdots) \,\cdot\, \text{times3}'(\cdots) \,\cdot\, \text{square}'(x).$$



Quando indiachiamo `require_grad=True` stiamo dicendo che avremo bisogno di calcolare un qualche gradiente rispetto alla variabile che stiamo istanziando. Da questo momento in poi, PyTorch terrà traccia di ogni operazione che dipende da questa variabile. L'intera storia con cui ricostruire le varie derivate è contenuta in quello che si chiama **computational graph**. Nel nostro caso:

<p>
<img src=https://github.com/CodingTomo/PyTorch-Tutorials/blob/master/Immagini/cg_1.png?raw=1 width="550" style="margin-left: auto;margin-right: auto;display: block;" />
</p>






Dopo aver calcolato la funzione obiettivo $y=3x^2+4$, è possibile chiamare il metodo `y.backward()` per calcolare le derivate di $y$ rispetto a tutte le variabili per cui l'attributo `require_grad` è True. Sempre nel nostro caso:

<p>
<img src=https://github.com/CodingTomo/PyTorch-Tutorials/blob/master/Immagini/cg_2.png?raw=1 width="550" style="margin-left: auto;margin-right: auto;display: block;" />
</p>

Osserviamo che i gradienti si **accumulano**. Bisogna quindi fare attenzione inizializzali quando necessario.

In [0]:
# Initialize x with some value
x = torch.tensor(2.0, requires_grad=True)
y = 3 * x**2  + 4

y.backward()
print("Il valore calcolato per la derivata è {}".format(x.grad))
print("Il valore atteso per la derivata è {}".format(6 * x))

print('-'*70)

y = 3 * x**2  + 4
y.backward()
print("Il valore calcolato per la derivata è {}".format(x.grad))
print("Il valore atteso per la derivata è {}".format(6 * x))

Con il metodo `detach()` possiamo **interrompere** la catena che tiene traccia di dipendenze dalle variabili già definite.

In [0]:
A = torch.rand(1,2, requires_grad=True)
B = A.mean()

print("B : ", B)
print("B.requires_grad :", B.requires_grad)

C = B.detach()

print("C : ", C)
print("C.requires_grad :", C.requires_grad)

**Esempio** *(retta tangente)* : Vogliamo trovare la retta tangente in un punto fissato ad una funzione espressa in coordinate polari della forma $f:[0,\infty) \times \mathbb{R} \rightarrow \mathbb{R}$ tale che $(\rho,\theta)\mapsto \rho - 2\sin(\theta)$.



1. Generiamo una griglia di punti utili a disegnare la funzione. Questo si può facilmente ottenere usando la funzione *meshgrid()*.
2. Passiamo i punti così generati in coordinate polari e calcoliamo il valore della funzione.
3. Calcoliamo il gradiente nel punto fissato e usiamolo per generare la retta tangente.

Infine, ricordiamo che:

$$
\bigg \{
\begin{array}{rl}
\rho = \sqrt{x^2+y^2} \\
\theta = \arctan\big(\frac{y}{x}\big) \\
\end{array}
$$







In [0]:
from matplotlib import pyplot as plt

def f(x, y):
    radius = torch.sqrt(x**2 + y**2)
    angle = torch.atan2(y, x)
    return radius-(2*torch.sin(angle))


def contour_plot():
    grid_x, grid_y = torch.meshgrid(torch.arange(-3, 3.1, 0.1), torch.arange(-3, 3.1, 0.1))
    outputs = f(grid_x, grid_y)
    
    plt.contourf(grid_x, grid_y, outputs, 15)
    plt.colorbar(label="f(x, y)")
    plt.xlabel("input x")
    plt.ylabel("input y")

contour_plot()
plt.show()

In [0]:
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)

z = f(x, y).backward()

df_dx = x.grad
df_dy = y.grad

def plot_tangent_line(x, y, df_dx, df_dy):
    x_points = torch.linspace(-3, 3, 100)
    y_points = - df_dx / df_dy * (x_points - x.detach()) + y.detach()
    plt.scatter(x.detach(), y.detach(), color='white')
    plt.plot(x_points, y_points, color='white')
    plt.xlim([-3, 3])
    plt.ylim([-3, 3])

    
contour_plot()
plot_tangent_line(x, y, df_dx, df_dy)

plt.show()

**Ossevazione**: il passaggio nel quale scriviamo la formula della retta tangente non è innoquo e fa uso del teorema della funzione implicita.

Autograd distingue due tipi di nodi all'interno del grafo dove memorizza la computazione. 


*   Le **foglie** sono i tensori definiti dal programmatore e su cui non è stata fatta alcuna operazione.
*   I **nodi** sono i tensori risultato di operazioni che possono anche, ma non necessariamente, coinvolgere altri tensori.



In [0]:
A = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
B = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True) + 2 
C = 5 * A 
print("A.is_leaf :", A.is_leaf)
print("B.is_leaf :", B.is_leaf)
print("C.is_leaf :", C.is_leaf)

La distinzione è utile da sapere in quanto Pytorch di default memorizza solo i gradienti dell'output rispetto alle foglie. Il risultato è memorizzato in `nome_foglia.grad`.



In [0]:
A = torch.Tensor([[1, 2], [3, 4]])
A.requires_grad_()

B = 5 * (A + 3)
C = B.mean()

print("A.grad :", A.grad)
print("B.grad :", B.grad)
C.backward()
print("\n-- Backward --\n")
print("A.grad :", A.grad)
print("B.grad :", B.grad)

Nella pratica spesso non è necessario, ma è comunque possibile memorizzare tutti i gradienti di tutti gli step usando la funzione `retain_grad()`.

In [0]:
A = torch.Tensor([[1, 2], [3, 4]])
A.requires_grad_()

B = 5 * (A + 3)
B.retain_grad()
C = B.mean()


print("A.grad :", A.grad)
print("B.grad :", B.grad)
C.backward()
print("\n-- Backward --\n")
print("A.grad :", A.grad)
print("B.grad :", B.grad)